In [4]:
import matplotlib.pyplot as plt
import mesa
from mesa.space import NetworkGrid
from mesa.time import RandomActivation
from mesa.time import SimultaneousActivation
import networkx as nx
import numpy as np

class BMIAgent(mesa.Agent):
    """
    An agent with BMI value
    """

    def __init__(self, unique_id, model, alpha, beta):
        super().__init__(unique_id, model)
        
        # Initialize each agents BMI from a ~ 15 + Gamma(alpha, beta)
        self.bmi = 15 + np.random.gamma(alpha, 1/beta)

    def step(self):
        """
        Step the model forward by one tick
        """
        # Get all agents connected in the graph to the current agent 
        neighbors = self.model.grid.get_neighbors(self.pos, include_center=False)
        # Calculate the mean BMI of all neighboring agents
        mean_bmi = np.mean([agent.bmi for agent in self.model.grid.iter_cell_list_contents(neighbors)])
        # The difference between agents BMI and mean of neighbors BMI
        dif = mean_bmi - self.bmi
        
        # If the difference is greater than the satificing radius adjust BMI
        if abs(dif) >= self.model.radius:
            # Step size amount to adjust agent BMI by
            step_size = min(abs(dif), 0.1)
            # Move down in BMI if agent heavier than neighbors
            if dif < 0:
                self.bmi -= step_size
            # Move up in BMI if agent lighter than neighbors
            if dif > 0:
                self.bmi += step_size

class FTAModel(mesa.Model):
    """
    A follow the average agent based model of BMI
        N: The number of agent
        rewire_prob: The probability of reqiring a node connection when 
                     initializing the Watts-Strogatz graph
        radius: The satisficing radius
        alpha: Gamma shape parameter for initializing agents BMI
        beta: Gamma parameterfor initializing agents BMI
        mean_degree: Number of initial neighbors each agent starts with
    """

    def __init__(self, N=100, rewire_prob=0.1, radius=0.1, alpha=3, beta=0.25, mean_degree=4):
        self.running = True
        # Number of agents in the model
        self.num_agents = N
        # Random activation scheduler
        self.schedule = SimultaneousActivation(self)
        # The satisficing radius
        self.radius = radius
        
        # Graph connecting the agents (using Watts-Strogatz algorithm)
        self.G = nx.watts_strogatz_graph(N, mean_degree, rewire_prob)
        self.grid = NetworkGrid(self.G)
        
        # Create agents
        for i, node in enumerate(self.G.nodes()):
            # Create agent i
            a = BMIAgent(i, self, alpha, beta)
            
            # Add agent to schedule and network
            self.schedule.add(a)
            self.grid.place_agent(a, node)
            
        self.datacollector = mesa.DataCollector(model_reporters={"Mean BMI": mean_bmi})

    def step(self):
        """Advance the model by one step."""
        self.datacollector.collect(self)
        self.schedule.step()
        
def mean_bmi(model):
    return np.mean([agent.bmi for agent in model.schedule.agents])

In [7]:
def run_model(n):
    model = FTAModel(N=n)
    for _ in range(200):
        model.step()
    return model

In [9]:
m = run_model(100)

In [12]:
m.datacollector.get_model_vars_dataframe()

,Mean BMI
0,26.023306
1,26.028306
2,26.034306
3,26.038306
4,26.043306
...,...
195,27.832306
196,27.832306
197,27.832306
198,27.832306


In [ ]:
Sim

In [28]:
data = mesa.batch_run(FTAModel, {"radius": [0, 0.4], "rewire_prob": [0.1, 0.9]}, 
                      max_steps=200, data_collection_period =1, iterations=10)

100%|███████████████████████████████████████████| 40/40 [00:32<00:00,  1.24it/s]


In [30]:
import pandas as pd
data = pd.DataFrame(data)

In [31]:
data.to_feather('mesa_data2.feather')

In [32]:
data

,RunId,iteration,Step,radius,rewire_prob,Mean BMI
0,0,0,0,0.0,0.1,27.831009
1,0,0,1,0.0,0.1,27.850292
2,0,0,2,0.0,0.1,27.870042
3,0,0,3,0.0,0.1,27.889292
4,0,0,4,0.0,0.1,27.910692
...,...,...,...,...,...,...
8035,39,9,196,0.4,0.9,28.047867
8036,39,9,197,0.4,0.9,28.046867
8037,39,9,198,0.4,0.9,28.044867
8038,39,9,199,0.4,0.9,28.042867
